In [1]:
import pandas as pd

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# load the data
df = pd.read_csv('data.csv', encoding='utf-8')

In [3]:
df.shape

(32063, 2)

In [4]:
# check the data for the first time
df.head()

,created_at,text
0,2018-07-25 07:43:00,RT @snas2030: خالد السليمان : لـ #ولي_العهد ال...
1,2018-07-25 07:41:20,RT @halimamuthffar: المختصر .. \n\nالعمل في ال...
2,2018-07-25 07:39:06,#الإسكان تطلق منصة جديدة يمكنك من خلالها الحص...
3,2018-07-25 07:39:05,#وزير_المالية : تم تخصيص 70 ألف أرض سكنية وتخص...
4,2018-07-25 07:39:05,تملك السعوديين للمنازل يقفز 50% والتصاعد مستم...


In [5]:
# clean the data from nonsense words, to help speed up the training process

from bs4 import BeautifulSoup
import re
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()


pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    
    words = tok.tokenize(clean)
    return (" ".join(words)).strip()
    

In [6]:
# applying the cleaning function

print("Cleaning and parsing the tweets...\n")
size = len(df)
clean_tweet_texts = []
for i in range(0, size):
    if (i+1)%10000 == 0:
        print("Tweets %d of %d has been processed" % (i+1, size))
    clean_tweet_texts.append(clean_str(df['text'].astype('U')[i]))

Cleaning and parsing the tweets...



/home/mgalalen/.local/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/JJ2GyjTjhP
#السعودية
#الغربيه
#اليمن
#رؤية_السعودية_2030
#المدينة_المنورة
#اليمن" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Tweets 10000 of 32063 has been processed
Tweets 20000 of 32063 has been processed


/home/mgalalen/.local/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/qVsRP9Ciea
#هويه_الحجاز." looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Tweets 30000 of 32063 has been processed


In [7]:
# add the cleaned data to the dataframe
df['clean_text'] = clean_tweet_texts

In [9]:
# view data after cleaning
df.head()

,created_at,text,clean_text
0,2018-07-25 07:43:00,RT @snas2030: خالد السليمان : لـ #ولي_العهد ال...,RT : خالد السليمان : لـ # ولي_العهد الأمير # م...
1,2018-07-25 07:41:20,RT @halimamuthffar: المختصر .. \n\nالعمل في ال...,RT : المختصر .. العمل في الشأن الثقافي ليس فقط...
2,2018-07-25 07:39:06,#الإسكان تطلق منصة جديدة يمكنك من خلالها الحص...,# الإسكان تطلق منصة جديدة يمكنك من خلالها الحص...
3,2018-07-25 07:39:05,#وزير_المالية : تم تخصيص 70 ألف أرض سكنية وتخص...,# وزير_المالية : تم تخصيص 70 ألف أرض سكنية وتخ...
4,2018-07-25 07:39:05,تملك السعوديين للمنازل يقفز 50% والتصاعد مستم...,تملك السعودين للمنازل يقفز 50 % والتصاعد مستمر...


## Data Annotation

In [10]:
# because the datasets it's large we consider automate the task of annotation using KMeans

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np

In [11]:
# vectorizing the words
tfidf = TfidfVectorizer()
data = tfidf.fit_transform(df['clean_text'].values.astype('U'))

In [14]:
# fitting data into KMeans and create 3 clusters
kmeans = KMeans(n_clusters=3)
kmeans.fit(data)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [17]:
# check the size of the data
len(kmeans.labels_.tolist()), data.shape, df.shape

(32063, (32063, 60440), (32063, 3))

In [18]:
df['label'] = kmeans.labels_.tolist()

In [19]:
df.head()

,created_at,text,clean_text,label
0,2018-07-25 07:43:00,RT @snas2030: خالد السليمان : لـ #ولي_العهد ال...,RT : خالد السليمان : لـ # ولي_العهد الأمير # م...,0
1,2018-07-25 07:41:20,RT @halimamuthffar: المختصر .. \n\nالعمل في ال...,RT : المختصر .. العمل في الشأن الثقافي ليس فقط...,0
2,2018-07-25 07:39:06,#الإسكان تطلق منصة جديدة يمكنك من خلالها الحص...,# الإسكان تطلق منصة جديدة يمكنك من خلالها الحص...,2
3,2018-07-25 07:39:05,#وزير_المالية : تم تخصيص 70 ألف أرض سكنية وتخص...,# وزير_المالية : تم تخصيص 70 ألف أرض سكنية وتخ...,2
4,2018-07-25 07:39:05,تملك السعوديين للمنازل يقفز 50% والتصاعد مستم...,تملك السعودين للمنازل يقفز 50 % والتصاعد مستمر...,2


In [27]:
# create empty dataframe for the cleaned data
clean_df = pd.DataFrame()
clean_df.head()
clean_df['text'] = df['clean_text']
clean_df['label'] = df['label']
clean_df.head()

,text,label
0,RT : خالد السليمان : لـ # ولي_العهد الأمير # م...,0
1,RT : المختصر .. العمل في الشأن الثقافي ليس فقط...,0
2,# الإسكان تطلق منصة جديدة يمكنك من خلالها الحص...,2
3,# وزير_المالية : تم تخصيص 70 ألف أرض سكنية وتخ...,2
4,تملك السعودين للمنازل يقفز 50 % والتصاعد مستمر...,2


In [ ]:
# show data count
print('Data labeled as 0 = {}'.format(clean_df[clean_df.label == 0].count()))
print('Data labeled as 1 = {}'.format(clean_df[clean_df.label == 1].count()))
print('Data labeled as 2 = {}'.format(clean_df[clean_df.label == 2].count()))

## save the cleaned data to the disk

In [22]:
# csv_file = 'cleaned_data.csv'
# clean_df.to_csv(csv_file,encoding='utf-8')
# my_df = pd.read_csv(csv_file,index_col=0)
# my_df.head()

,text,label
0,RT : خالد السليمان : لـ # ولي_العهد الأمير # م...,0
1,RT : المختصر .. العمل في الشأن الثقافي ليس فقط...,0
2,# الإسكان تطلق منصة جديدة يمكنك من خلالها الحص...,2
3,# وزير_المالية : تم تخصيص 70 ألف أرض سكنية وتخ...,2
4,تملك السعودين للمنازل يقفز 50 % والتصاعد مستمر...,2


## Building the model

In [28]:
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import numpy as np

In [29]:
# vectorize the data 
cv = CountVectorizer()
X_data = cv.fit_transform(my_df['text'].values.astype('U'))

In [30]:
# To find importance of the word with respect to the corpus.
# see: http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

tfidf = TfidfTransformer()
X_data_tfidf = tfidf.fit_transform(X_data)

In [31]:
# segment the data to train and test sets

X_train, X_test, y_train, y_test = train_test_split(X_data_tfidf, my_df['label'], test_size=0.33, random_state=42)

In [32]:
X_train.shape, X_test.shape

((21482, 60440), (10581, 60440))

## SVM Model

In [33]:
svc = SVC()
svc.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [36]:
preds = svc.predict(X_test)
acc = np.mean(preds == y_test)
print('SVC model accuracy: {}'.format(acc*100))

SVC model accuracy: 86.45685662980814


In [38]:
cr = classification_report(y_test, svc.predict(X_test))
print(cr)

             precision    recall  f1-score   support

          0       0.86      1.00      0.93      9148
          1       0.00      0.00      0.00      1157
          2       0.00      0.00      0.00       276

avg / total       0.75      0.86      0.80     10581



/home/mgalalen/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## SGD Model

In [39]:
# optimized version of SVM
from sklearn.linear_model import SGDClassifier

In [40]:
clf = SGDClassifier()
clf.fit(X_train, y_train)

/home/mgalalen/.local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [41]:
preds = clf.predict(X_test)
acc = np.mean(preds == y_test)
print('SGD model accuracy: {}'.format(acc*100))

SGD model accuracy: 99.79208014365372


In [42]:
cr = classification_report(y_test, clf.predict(X_test))
print(cr)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      9148
          1       1.00      0.98      0.99      1157
          2       1.00      1.00      1.00       276

avg / total       1.00      1.00      1.00     10581



In [43]:
# SGD is faster and better than SVC